# Units

In ***Synthesizer*** all quantities a user interacts with (that are not dimensionless) have units associated to them to avoid confusion. We implement this functionality with the ***unyt*** package. 

When interfacing with ***Synthesizer*** objects and methods a user should provide quantites with units attached to them. This can be easily achieved with the ***unyt*** package.

In [1]:
from unyt import Mpc


# Define a variable with units 
x = 1 * Mpc

print(x)
print("x is now a unyt_quantity: type(x)=", type(x))

1 Mpc
x is now a unyt_quantity: type(x)= <class 'unyt.array.unyt_quantity'>


All unit functionality in ***Synthesizer*** is house in the `units` module. In this module we have defined a dictionary containing the default units of all attributes throughout ***Synthesizer***.

In [2]:
from synthesizer.units import default_units


print(default_units)

{'lam': Å, 'obslam': Å, 'wavelength': Å, 'original_lam': Å, 'lam_min': Å, 'lam_max': Å, 'lam_eff': Å, 'lam_fwhm': Å, 'mean_lams': Å, 'pivot_lams': Å, 'nu': Hz, 'obsnu': Hz, 'nuz': Hz, 'original_nu': Hz, 'luminosity': erg/s, 'luminosities': erg/s, 'bolometric_luminosity': erg/s, 'bolometric_luminosities': erg/s, 'lnu': erg/(Hz*s), 'llam': erg/(s*Å), 'continuum': erg/(Hz*s), 'flux': erg/(cm**2*s), 'fnu': nJy, 'ew': Å, 'coordinates': Mpc, 'smoothing_lengths': Mpc, 'softening_length': Mpc, 'velocities': km/s, 'mass': unyt_quantity(1., 'Msun'), 'masses': unyt_quantity(1., 'Msun'), 'initial_masses': unyt_quantity(1., 'Msun'), 'initial_mass': unyt_quantity(1., 'Msun'), 'current_masses': unyt_quantity(1., 'Msun'), 'dust_masses': unyt_quantity(1., 'Msun'), 'ages': yr, 'accretion_rate': unyt_quantity(1., 'Msun/yr'), 'accretion_rates': unyt_quantity(1., 'Msun/yr'), 'bol_luminosity': erg/s, 'bb_temperature': K, 'bb_temperatures': K, 'inclination': degree, 'inclinations': degree, 'resolution': Mpc,

As you can see, most units are defined symbollically, however, you may notice units such as `Msun` (the default used for masses) is defined as a compound unit in terms of `kg`s. Shortly we will cover working with the quantities returned by ***Synthesizer*** and how compound units work in practice.

## The `Units` object

The unit system is defined by the `Units` object. This object contains a collection of attributes defining the units associated to each quantity throughout ***Synthesizer*** which is not dimensionless. Importantly, `Units` is a `Singleton` object. This means there can only ever be one instance of `Units`, if a second is instantiated then the first is returned. This ensures that the unit system remains consistent when running ***Synthesizer***.

In [3]:
from synthesizer.units import Units

# Define multiple Units instances
units1 = Units()
units2 = Units()

print("Both units instances are the same object:", units1 is units2)

Both units instances are the same object: True


You can take a look at the unit system by printing the instance of `Units`.

In [4]:
print(units1)

Unit System: 
lam:                Å
obslam:             Å
wavelength:         Å
original_lam:       Å
lam_min:            Å
lam_max:            Å
lam_eff:            Å
lam_fwhm:           Å
mean_lams:          Å
pivot_lams:         Å
nu:                 Hz
obsnu:              Hz
nuz:                Hz
original_nu:        Hz
luminosity:         erg/s
luminosities:       erg/s
bolometric_luminosity: erg/s
bolometric_luminosities: erg/s
lnu:                erg/(Hz*s)
llam:               erg/(s*Å)
continuum:          erg/(Hz*s)
flux:               erg/(cm**2*s)
fnu:                nJy
ew:                 Å
coordinates:        Mpc
smoothing_lengths:  Mpc
softening_length:   Mpc
velocities:         km/s
mass:               0.9999999999999999 Msun
masses:             0.9999999999999999 Msun
initial_masses:     0.9999999999999999 Msun
initial_mass:       0.9999999999999999 Msun
current_masses:     0.9999999999999999 Msun
dust_masses:        0.9999999999999999 Msun
ages:               yr
accret

## Modifying the default unit system

If the default unit system works for your needs then you don't need to do anything. You will never interact with the `Units` object and all quantites will have the default units associated to them automatically. However, if you need to change one or more of the units used you can import `Units` and instantiate it with a dictionary of the modified quantities.


In [5]:
from unyt import kpc, Myr


# Make the dictionary containing the units we want to change
new_units = {'coordinates': kpc, 'smoothing_lengths': kpc, 'softening_length': kpc, 'ages': Myr}

# Set up the modified unit system
units = Units(new_units)

print()
print(units)

WARNING! Units are already set. 
Any modified units will not take effect. 
Units should be configured before running anything else... 
but you could (and shouldn't) force it: Units(new_units_dict, force=True).

Unit System: 
lam:                Å
obslam:             Å
wavelength:         Å
original_lam:       Å
lam_min:            Å
lam_max:            Å
lam_eff:            Å
lam_fwhm:           Å
mean_lams:          Å
pivot_lams:         Å
nu:                 Hz
obsnu:              Hz
nuz:                Hz
original_nu:        Hz
luminosity:         erg/s
luminosities:       erg/s
bolometric_luminosity: erg/s
bolometric_luminosities: erg/s
lnu:                erg/(Hz*s)
llam:               erg/(s*Å)
continuum:          erg/(Hz*s)
flux:               erg/(cm**2*s)
fnu:                nJy
ew:                 Å
coordinates:        Mpc
smoothing_lengths:  Mpc
softening_length:   Mpc
velocities:         km/s
mass:               0.9999999999999999 Msun
masses:             0.9999999999999999

Something has gone wrong... but recall that the unit system will return the original if one exists, so actually this should be completely expected.

This issue highlights the need to set up `Units` **before** doing anything else. If any computations have been done the `Units` instance will exist and will not be modifiable after the fact. However, should you fall in this trap the code will warn you as above, no hidden gotchas here!

Now, lets go against the advise above and use the **highly inadviasble** force argument to get a new Unit system but note, **in a real use case forcing a modified unit system WILL NOT convert existing quantities to the new unit system**.

In [6]:
# Set up the modified unit system
units = Units(new_units, force=True)

print()
print(units)

Redefining unit system:
coordinates: kpc
smoothing_lengths: kpc
softening_length: kpc
ages: Myr

Unit System: 
lam:                Å
obslam:             Å
wavelength:         Å
original_lam:       Å
lam_min:            Å
lam_max:            Å
lam_eff:            Å
lam_fwhm:           Å
mean_lams:          Å
pivot_lams:         Å
nu:                 Hz
obsnu:              Hz
nuz:                Hz
original_nu:        Hz
luminosity:         erg/s
luminosities:       erg/s
bolometric_luminosity: erg/s
bolometric_luminosities: erg/s
lnu:                erg/(Hz*s)
llam:               erg/(s*Å)
continuum:          erg/(Hz*s)
flux:               erg/(cm**2*s)
fnu:                nJy
ew:                 Å
coordinates:        kpc
smoothing_lengths:  kpc
softening_length:   kpc
velocities:         km/s
mass:               0.9999999999999999 Msun
masses:             0.9999999999999999 Msun
initial_masses:     0.9999999999999999 Msun
initial_mass:       0.9999999999999999 Msun
current_masses:     

## Working with `Quantity`s

As previously mentioned, there is no need to work with the `Units` object itself beyond intially defining a modified unit system. Beyond this all unit operations are handled "behind the scenes". This hidden functionality is enabled by the `Qunatity` object. 

All attributes on ***Synthesizer*** objects which carry units are in fact `Quantity`s. `Quantitiy`s carry a reference to the global unit system and extract the appropriate units depending on the name of the variable storing the `Quantity`. As such a user will never instantiate a quantity themselves but there usage is important. 

The one simple thing to keep in mind is how to return the value with or without units. This is achieved by the application or omission of a leading underscore to a variable name. 

Lets create an `Sed` which has a wavelength array stored under `lam`. Note that since we are not explictly passing units the default unit system is assumed.

In [7]:
import numpy as np
from synthesizer.sed import Sed


# Make an sed with arbitrary arguments
sed = Sed(lam=np.linspace(10, 1000, 10), lnu=np.ones(10))

We can access this attribute with units as you would expect to access any attribute.

In [8]:
print(sed.lam)

[  10.  120.  230.  340.  450.  560.  670.  780.  890. 1000.] Å


Or we can append a leading underscore and return it without units.

In [9]:
print(sed._lam)

[  10.  120.  230.  340.  450.  560.  670.  780.  890. 1000.]


In the case of compound units this is marginly messier. Lets demonstrate with some `Stars`.

In [10]:
from synthesizer.particle.stars import Stars


# Create a dummy Stars object
stars = Stars(initial_masses=np.random.rand(10), ages=np.ones(10), metallicities=np.ones(10))

If we print the `initial_masses` with units we get the compound version in `kg`.

In [11]:
print(stars.initial_masses)

[0.83081941 0.29035634 0.93979554 0.92722725 0.78710397 0.91181419
 0.74860817 0.5618026  0.8840734  0.37958358] Msun


However, if we extract the values alone we get the values we expect in $M_\odot$.

In [12]:
print(stars._initial_masses)

[0.83081941 0.29035634 0.93979554 0.92722725 0.78710397 0.91181419
 0.74860817 0.5618026  0.8840734  0.37958358]


Its worth keeping this in mind whenever extracting masses from ***Synthesizer*** objects.

### Automatic unit conversion

Finally lets utilise some automatic unit conversion. If we have a mixture of properties to input to a ***Synthesizer*** object, all with different units to the global unit system, we don't have to convert them all before inputting them. As long as we pass them to ***Synthesizer*** with ***unyt*** units attached, the conversion will be handled automatically. Here we use a Stars object again to demonstrate.

In [13]:
from unyt import Mpc, g, m


# Create a dummy Stars object
stars = Stars(initial_masses=np.random.rand(10) * 10 ** 34. * g, 
              ages=np.ones(10) * Myr, 
              metallicities=np.ones(10), 
              coordinates=np.random.rand(10, 3) * Mpc, 
              smoothing_lengths=np.random.rand(10) * 10**22. * m)

print("stars.initial_masses[0]=", stars.initial_masses[0], "=", stars._initial_masses[0], "Msun")
print("stars.ages[0]=", stars.ages[0])
print("stars.coordinates[0]=", stars.coordinates[0])
print("stars.smoothing_lengths[0]=", stars.smoothing_lengths[0])

stars.initial_masses[0]= 2.002398921183607 Msun = 2.0023989211836075 Msun
stars.ages[0]= 1000000.0 yr
stars.coordinates[0]= [0.621627   0.44052395 0.73576801] Mpc
stars.smoothing_lengths[0]= 0.2796711344242985 Mpc
